Щоб запустити наступні записники, додайте персональний токен GitHub у `.env` як `GITHUB_TOKEN`. Токен потрібен для використання GitHub Models через Azure AI Inference.

## Покрокова інструкція для практичної роботи №5

1. Перевірте `.env`, щоб містив оновлений `GITHUB_TOKEN` з доступом `models:read`, та перезапустіть ядро перед запуском ноутбука.
2. Запустіть комірку з `pip install`, щоб переконатися, що інстальовані `azure-ai-inference` та `python-dotenv`.
3. По черзі виконуйте блоки, обов'язково зберігайте локальну копію `embedding_index_githubmodels.json` для повторних запусків.
4. Зафіксуйте власні запити і результати у звіті: вони будуть потрібні для індивідуального завдання та захисту.
5. Порівняйте поведінку цього ноутбука з AIMLAPI-варіантом, як вимагають методичні рекомендації.


In [1]:
%pip install azure-ai-inference python-dotenv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from azure.ai.inference import EmbeddingsClient
from azure.core.credentials import AzureKeyCredential

load_dotenv()

token = os.getenv("GITHUB_TOKEN", "")
assert token, "ERROR: GITHUB_TOKEN is missing. Будь ласка, додайте його у .env або середовище."

endpoint = "https://models.inference.ai.azure.com"
embed_model_name = "text-embedding-3-small"

SIMILARITY_THRESHOLD = 0.3
DATASET_PATH = Path("../embedding_index_3m.json")
CACHE_PATH = Path("../embedding_index_githubmodels.json")
BATCH_SIZE = 32

embed_client = EmbeddingsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

Далі завантажимо індекс відео у `pandas` DataFrame та переконаємось, що дані готові до побудови векторів. Після виконання цієї комірки перегляньте розмірність набору даних, щоб описати його у звіті.


In [3]:
def load_dataset(source: Path) -> pd.DataFrame:
    df = pd.read_json(source).fillna("")
    return df.drop(columns=["ada_v2"], errors="ignore")

Наступна функція створює або завантажує ембедінги за допомогою GitHub Models. Перший запуск може тривати кілька хвилин, адже потрібно обробити всі записи.

> **Нотатка для звіту:** зафіксуйте час побудови індексу та порівняйте його з AIMLAPI-рішенням. Якщо використовується кеш, опишіть, як він пришвидшує повторні запуски.


In [4]:
def build_or_load_github_index(df: pd.DataFrame, cache_path: Path | None = None, batch_size: int = 32) -> pd.DataFrame:
    if cache_path and cache_path.exists():
        cached_df = pd.read_json(cache_path).fillna("")
        if "github_embedding" in cached_df.columns:
            print(f"Завантажено вбудовування з кешу: {cache_path}")
            return cached_df
        else:
            print("Кеш знайдено, але без стовпчика 'github_embedding'. Створюємо індекс заново.")

    summaries = df["summary"].tolist()
    embeddings: list[list[float]] = []

    total = len(summaries)
    for start in range(0, total, batch_size):
        batch = summaries[start:start + batch_size]
        response = embed_client.embed(
            input=batch,
            model=embed_model_name,
        )
        embeddings.extend([item.embedding for item in response.data])
        done = min(start + batch_size, total)
        print(f"Оброблено {done} / {total} записів", end="\r")

    print()
    enriched_df = df.copy()
    enriched_df["github_embedding"] = embeddings

    if cache_path:
        enriched_df.to_json(cache_path, orient="records")
        print(f"Індекс збережено до {cache_path}")

    return enriched_df


In [5]:
videos_df = load_dataset(DATASET_PATH)
videos_df = build_or_load_github_index(videos_df, CACHE_PATH, batch_size=BATCH_SIZE)
videos_df.head()

Оброблено 1409 / 1409 записів
Індекс збережено до ../embedding_index_githubmodels.json


,speaker,title,videoId,start,seconds,summary,github_embedding
0,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:00:00,0,Join Seth Juarez as he discusses ethical conce...,"[0.05830316, 0.0032841517, 0.017757142, 0.0241..."
1,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:03:07,187,"In this video, the speaker discusses the chall...","[0.034540925, 0.010548099, 0.04061352, 0.00216..."
2,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:06:13,373,The video discusses the limitations of general...,"[0.027302632, 0.026398426, 0.040799573, -0.000..."
3,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:09:21,561,The video discusses the importance of consider...,"[0.045100074, -0.00481684, 0.030940464, 0.0169..."
4,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:12:24,744,The video discusses the importance of understa...,"[0.017057244, 0.029829111, 0.03688313, 0.02129..."


Додамо функції для обчислення косинусної подібності та отримання ембедінга запиту. Ці блоки можна винести у власні модулі, якщо ви будуєте продакшн-рішення. Додайте короткий опис алгебри косинусної подібності у звіт.


In [6]:
def cosine_similarity(vec_a, vec_b) -> float:
    a = np.array(vec_a)
    b = np.array(vec_b)
    denom = np.linalg.norm(a) * np.linalg.norm(b)
    if denom == 0:
        return 0.0
    return float(np.dot(a, b) / denom)

def get_query_embedding(query: str) -> list[float]:
    response = embed_client.embed(
        input=[query],
        model=embed_model_name,
    )
    return response.data[0].embedding


Функція `get_videos` поверне найбільш релевантні відео для запиту. Налаштуйте параметр `rows`, щоб отримати різну кількість результатів, та проаналізуйте відмінності у відборі.


In [7]:
def get_videos(query: str, video_vectors: pd.DataFrame, rows: int = 5, threshold: float | None = SIMILARITY_THRESHOLD) -> pd.DataFrame:
    query_embedding = get_query_embedding(query)
    scored = video_vectors.copy()
    scored["similarity"] = scored["github_embedding"].apply(lambda emb: cosine_similarity(query_embedding, emb))
    scored = scored.sort_values("similarity", ascending=False)
    if threshold is not None:
        scored = scored[scored["similarity"] >= threshold]
    return scored.head(rows)


Допоміжна функція красиво виводить результати пошуку. Ви можете замінити її на власний формат (наприклад, таблицю або JSON), якщо того потребує ваше індивідуальне завдання.


In [8]:
def display_results(videos: pd.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        return f"https://youtu.be/{video_id}?t={seconds}"

    if videos.empty:
        print(f"\nНе знайдено результатів для запиту: '{query}'. Спробуйте змінити запит або зменшити поріг схожості.")
        return

    print(f"\nВідео, схожі на '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], int(row["seconds"]))
        print(f" - {row['title']}")
        print(f"   Резюме: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Схожість: {row['similarity']:.3f}")
        print(f"   Доповідачі: {row['speaker']}")


Спробуйте пошук за прикладом, а потім підставте власні запити. Для звіту додайте принаймні два запити з різними темами та поясніть, наскільки результати задовольняють інформаційну потребу.


In [9]:
sample_query = "Що таке ONNX?"
sample_videos = get_videos(sample_query, videos_df, rows=5)
display_results(sample_videos, sample_query)



Відео, схожі на 'Що таке ONNX?':
 - AI Show | Scikit-Learn with Andreas Müller
   Резюме: ONNX is a serialization format developed by Microsoft, Nvidia, and other companies that allows for...
   YouTube: https://youtu.be/ZEq0ivR_pzg?t=1109
   Схожість: 0.817
   Доповідачі: Andreas Müller, Seth Juarez
 - Faster and Lighter Model Inference with ONNX Runtime from Cloud to Client
   Резюме: ONNX Runtime, a high-performance inference engine, is being used by a variety of popular frameworks...
   YouTube: https://youtu.be/WDww8ce12Mc?t=184
   Схожість: 0.745
   Доповідачі: Emma
 - ONNX Runtime speeds up Image Embedding model in Bing Semantic Precise Image Search
   Резюме: In this episode of the AI Show, Vinitra Swamy from the ONNX engineering team at...
   YouTube: https://youtu.be/pmb6cjngbcA?t=0
   Схожість: 0.714
   Доповідачі: Vinitra Swamy
 - ONNX Runtime speeds up Image Embedding model in Bing Semantic Precise Image Search
   Резюме: The video discusses the ONNX Runtime, an open-sour

---
### Поради щодо індивідуального завдання

- Збережіть ембедінги у кеш-файл і використовуйте його для експериментів з різними запитами, щоб не перевищувати ліміти GitHub Models.
- Для статистичних підзавдань (графіки, частоти, кластери) можна використовувати `pandas`, `matplotlib` або `scikit-learn` у додаткових комірках цього ноутбука.
- Документуйте параметри запитів, фільтри та метрики, які використовуєте, щоб легко захистити роботу.
- Порівняйте результати з AIMLAPI-ноутбуком та опишіть відмінності у якості, швидкості та вартості.
